**Specifying the Target :**

Both methods supply the “target” of the conflict using either the n*amed constraint* or by *column inference*:


 - 1 : The *Insert.on_conflict_do_update.index_elements* argument specifies a sequence containing string column names, Column objects, and/or SQL expression elements, which would *identify a unique index* : 

In [1]:
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import Table, Column, MetaData, Integer, String, create_engine, URL


url = URL.create(
    drivername="postgresql",
    username="postgres",
    password="1234",
    host="192.168.0.8",
    port="5432",
    database="tests"
)

engine = create_engine(url)

metadata = MetaData()
table = Table(
    "data",
    metadata,
    Column('id', Integer, primary_key=True),
    Column('data', String)
)

insert_stmt = insert(table).values(
    id= 2, # some_existing_id
    data='some new data') # inserted value

do_update_stmt = insert_stmt.on_conflict_do_update(
    index_elements=['id'],
    set_=dict(data='updated value')
)
print(do_update_stmt)
do_update_stmt = insert_stmt.on_conflict_do_update(
    index_elements=[table.c.id],
    set_=dict(data='updated value')
)
print(do_update_stmt)

INSERT INTO data (id, data) VALUES (%(id)s, %(data)s) ON CONFLICT (id) DO UPDATE SET data = %(param_1)s
INSERT INTO data (id, data) VALUES (%(id)s, %(data)s) ON CONFLICT (id) DO UPDATE SET data = %(param_1)s


 - 2: When using Insert.on_conflict_do_update.index_elements to infer an index, a partial index can be inferred by also specifying the use the *Insert.on_conflict_do_update.index_where* parameter:

In [ ]:

user_table = Table(
    "user_table",
    metadata,
    Column('id', Integer, primary_key=True),
    Column('user_email', String), # try : index= True and primary_key=True 
    Column('data', String),
    extend_existing=True
)
user_table.create(engine)

with engine.connect() as conn :
    conn.execute(user_table.insert(), 
                 [{'id': 1, "data": "some data 1", "user_email": 'a@b.com'},
                   {'id': 2,"data": "some data 2", "user_email": 'a@gmail.com'}])
    conn.commit()
    

stmt = insert(user_table).values(user_email='a@b.com', data='inserted data')
stmt = stmt.on_conflict_do_update(
    index_elements=[user_table.c.user_email],
    index_where=user_table.c.user_email.like('%@gmail.com'),
    set_=dict(data=stmt.excluded.data)
)
print(stmt)

with engine.begin() as conn :
    conn.execute(stmt)

# ME : آیا مقدار index_elements باید به صورت یونیک در تعریف جدول وجود داشته باشه ؟ 
# why i'm getting : InvalidColumnReference: there is no unique or exclusion constraint matching the ON CONFLICT specification

# ME : index_where چی کار میکند ؟ 

- 3: The *Insert.on_conflict_do_update.constraint* argument is used to specify an index directly rather than inferring it. This can be the name of a UNIQUE constraint, a PRIMARY KEY constraint, or an INDEX:

In [23]:
do_update_stmt = insert_stmt.on_conflict_do_update(
    constraint='my_table_idx_1',
    set_=dict(data='updated value')
)
print(do_update_stmt)
do_update_stmt = insert_stmt.on_conflict_do_update(
    constraint='data_pkey',
    set_=dict(data='updated value')
)
print(do_update_stmt)

with engine.begin() as conn:
    conn.execute(do_update_stmt)

INSERT INTO data (id, data) VALUES (%(id)s, %(data)s) ON CONFLICT ON CONSTRAINT my_table_idx_1 DO UPDATE SET data = %(param_1)s
INSERT INTO data (id, data) VALUES (%(id)s, %(data)s) ON CONFLICT ON CONSTRAINT data_pkey DO UPDATE SET data = %(param_1)s


- 4: The Insert.on_conflict_do_update.constraint argument may also refer to a SQLAlchemy construct representing a constraint, e.g. UniqueConstraint, PrimaryKeyConstraint, Index, or ExcludeConstraint. In this use, if the constraint has a name, it is used directly. Otherwise, if the constraint *is unnamed*, then inference will be used, where the expressions and optional WHERE clause of the constraint will be spelled out in the construct. This use is especially convenient to refer to the named or unnamed primary key of a Table using the *Table.primary_key* attribute:

In [26]:
do_update_stmt = insert_stmt.on_conflict_do_update(
    constraint=table.primary_key,
    set_=dict(data='updated value')
)
print(do_update_stmt)

# ME : حالات دیگر ؟ یحتمل خود شی های UniqueConstraint ,...

INSERT INTO data (id, data) VALUES (%(id)s, %(data)s) ON CONFLICT (id) DO UPDATE SET data = %(param_1)s
